## RQ1 Data Preparation

[Add description]

Basic steps:
- Load all datasets (5 forest definitions and Natura 2000)
- Filter/clip to German Natura 2000


In [11]:
## Some initial setup

# Setting the current working directory
from pathlib import Path
cwd = Path.cwd()
#print(cwd)

### Load Datasets

As several of the datasets require login credentials and are not available through API, I decided to download all the required data and simply load it into Python.

Data required:
- NASA Land Cover
- ESA Land Cover
- JAXA FNF 
- CORINE Land Use
- German Land Use
- Natura 2000 protected areas


In [12]:
### JAXA
# Downloaded from: 

### CORINE
# Downloaded from: https://land.copernicus.eu/en/products/corine-land-cover/clc2018#download

